In [2]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer,  text_to_word_sequence
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback, ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed, Dropout
from keras import backend as K
from keras import optimizers
from keras.models import Model
import nltk
import re
import matplotlib.pyplot as plt
import sys
from sklearn.metrics import roc_auc_score
from nltk import tokenize
import seaborn as sns
import keras

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 3s 0us/step


In [0]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=150, padding='post')
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=150, padding='post')

In [0]:
word2idx = keras.datasets.imdb.get_word_index()

In [0]:
idx2word = {v:k for k, v in word2idx.items()}

In [0]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatibl|e with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [4]:
att = AttentionWithContext()

In [5]:
input_ = keras.layers.Input(shape=(150,))
x = keras.layers.Embedding(10000+1, 300, mask_zero=True)(input_)
x = keras.layers.GRU(256, return_sequences=True)(x)
x = att(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model = keras.models.Model(input_, x)

In [7]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 300)          3000300   
_________________________________________________________________
gru_1 (GRU)                  (None, 150, 256)          427776    
_________________________________________________________________
attention_with_context_1 (At (None, 256)               66048     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 3,527,149
Trainable params: 3,527,149
Non-trainable params: 0
_________________________________________________

In [23]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [24]:
hist = model.fit(x_train, y_train, batch_size=256, epochs=50, validation_data=(x_test, y_test))




Train on 25000 samples, validate on 25000 samples
Epoch 1/50





25000/25000 [==============================] - 46s 2ms/step - loss: 0.4459 - acc: 0.7928 - val_loss: 0.2928 - val_acc: 0.8801
Epoch 2/50
25000/25000 [==============================] - 36s 1ms/step - loss: 0.2210 - acc: 0.9153 - val_loss: 0.3098 - val_acc: 0.8684
Epoch 3/50
25000/25000 [==============================] - 35s 1ms/step - loss: 0.1567 - acc: 0.9438 - val_loss: 0.3353 - val_acc: 0.8614
Epoch 4/50
25000/25000 [==============================] - 35s 1ms/step - loss: 0.1282 - acc: 0.9550 - val_loss: 0.4019 - val_acc: 0.8582
Epoch 5/50
25000/25000 [==============================] - 35s 1ms/step - loss: 0.0886 - acc: 0.9694 - val_loss: 0.5112 - val_acc: 0.8536
Epoch 6/50
25000/25000 [==============================] - 35s 1ms/step - loss: 0.0594 - acc: 0.9806 - val_loss: 0.5245 - val_acc: 0.8464
Epoch 7/50
25000/25000 [==============================] - 35s 1ms/step - loss: 0.0378 - acc: 0.9884 - val_loss: 0.6930 -